# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Inteligencia Artificial y Aprendizaje Automático**
### Tecnológico de Monterrey
### Prof. Luis Eduardo Falcón Morales

## **Actividad de la Semana 6**
### **Árboles de decisión y bosque aleatorio.**

**Nombres y matrículas de los integrantes del equipo:**

* Jonathan Garza Bennet | 
* Diego Fernando Guerra Burgos | 
* Fernando Maytorena Espinosa de los Monteros | A01110946 
* José Francisco Miranda García |  

## Riesgo Crediticio
El asignar un crédito sabemos que conlleva un riesgo para el prestamista en caso de que el deudor no 
pague al final la cantidad asignada, o inclusive el equivocarnos al negarle el préstamo a alguien que sí 
era confiable. Durante décadas se ha tratado de resolver dicho problema desde muchas áreas del 
conocimiento y en particular las técnicas de Aprendizaje Automático (Machine Learning) han brindado 
y siguen proporcionando nuevas formas de enfrentar este problema.

No existen muchas bases de datos abiertas bien documentadas sobre este problema, sin embargo los 
datos del archivo SouthGermanCredit.asc del South_German_Credit_Data_Set de la página de la UCI es 
una sobre la cual se hace mucha investigación en torno a minimizar el riesgo en la asignación de 
créditos. En esta tarea se trabajará con dichos datos y los puedes encontrar dentro del archivo zip de la 
siguiente liga:  https://archive.ics.uci.edu/ml/datasets/South+German+Credit

En la página de la liga anterior también se encuentra el archivo codetable.txt en el archivo zip. Ahí 
puedes encontrar más información detallada sobre el significado y tipo de cada variable. 
Al menos en las siguientes ligas puedes encontrar mayor información de dichos datos: 
- https://www.kaggle.com/competitions/south-german-credit-prediction/overview  
- https://www.semanticscholar.org/paper/South-German-Credit-Data-Classification-Using-to-Religia-Pranoto/219c9968cfa2cbb802376cf88035fe5e664b4418  
- https://ieeexplore.ieee.org/document/9239944  

Así, con base a datos históricos, el objetivo es predecir si una persona es clasificada como confiable a la 
asignación de un crédito, o no lo es. La variable de salida se llama “kredit”.  

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [133]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import sklearn.pipeline as Pipeline
import sklearn.compose as compose
import sklearn.impute as impute
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# **Ejercicio-1.** 

Carga los datos y sustituye los nombres de las columnas del alemán al inglés de acuerdo a como 
se indica en la página de la UCI.

NOTA: Si lo deseas, puedes traducirlos y ponerlas en español.

In [134]:
# Primero, cargamos los datos utilizando la carpeta zip que se describe en las instrucciones.
# zip_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00522/SouthGermanCredit.zip'

df = pd.read_csv("/content/sample_data/SouthGermanCredit/SouthGermanCredit.asc", sep=' ')
df.head()

,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [135]:
#Para poder tener el archivo desde github sin tenerlo local, lo subimos a github en forma publica, y le damos click en el boton "raw" y nos da la dirección, recordar que tiene que estar publico
#df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SouthGermanCredit.asc', sep='\s+')
#df= pd.read_csv('https://github.com/a01110946/iayaa-Equipo-50/blob/main/SouthGermanCredit.asc.txt', sep='\s')
# df.head()

In [136]:
# Después de cargar los datos, reemplazamos los nombres de las columnas de alemán a inglés, siguiendo los nombres proporcionados en el archivo de descripción.
df.rename(columns = {
    'laufkont':'status',
    'laufzeit':'duration',
    'moral':'credit_history',
    'verw':'purpose',
    'hoehe':'amount',
    'sparkont':'savings',
    'beszeit':'employment_duration',
    'rate':'installment_rate',
    'famges':'personal_status_sex',
    'buerge':'other_debtors',
    'wohnzeit':'present_residence',
    'verm':'property',
    'alter':'age',
    'weitkred':'other_installment_plans',
    'wohn':'housing',
    'bishkred':'number_credits',
    'beruf':'job',
    'pers':'people_liable',
    'telef':'telephone',
    'gastarb':'foreign_worker',
    'kredit':'credit_risk'
}, inplace = True)

df.describe()
# Con el método describe() podemos ver las estadísticas básicas de cada columna numérica; y aprovechamos para ver los nombres de las columnas en inglés. 

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,2.58,20.90,2.54,2.83,3271.25,2.10,3.38,2.97,2.68,1.15,...,2.36,35.54,2.67,1.93,1.41,2.90,1.84,1.40,1.96,0.70
std,1.26,12.06,1.08,2.74,2822.75,1.58,1.21,1.12,0.71,0.48,...,1.05,11.35,0.71,0.53,0.58,0.65,0.36,0.49,0.19,0.46
min,1.00,4.00,0.00,0.00,250.00,1.00,1.00,1.00,1.00,1.00,...,1.00,19.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00
25%,1.00,12.00,2.00,1.00,1365.50,1.00,3.00,2.00,2.00,1.00,...,1.00,27.00,3.00,2.00,1.00,3.00,2.00,1.00,2.00,0.00
50%,2.00,18.00,2.00,2.00,2319.50,1.00,3.00,3.00,3.00,1.00,...,2.00,33.00,3.00,2.00,1.00,3.00,2.00,1.00,2.00,1.00
75%,4.00,24.00,4.00,3.00,3972.25,3.00,5.00,4.00,3.00,1.00,...,3.00,42.00,3.00,2.00,2.00,3.00,2.00,2.00,2.00,1.00
max,4.00,72.00,4.00,10.00,18424.00,5.00,5.00,4.00,4.00,3.00,...,4.00,75.00,3.00,3.00,4.00,4.00,2.00,2.00,2.00,1.00


# **Ejercicio-2.**

Realiza una partición de los datos en el conjunto de entrenamiento del 85% y el de prueba de 15%. Los modelos se estarán entrenando con el método de validación cruzada, así que no es necesario en este paso generar el conjunto de validación. Define como la variable X a todas las variables de entrada y a la variable Y como la variable de salida. 

In [137]:
X = df.iloc[:,0:20]
y = df.iloc[:, -1]
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.85, random_state=10)
X_train.head()
df.columns

print("Registros de Entrenamiento: \n" + str(X_train.columns) + "\n")
print("Registros de Validación: \n" + str(X_val.columns))

Registros de Entrenamiento: 
Index(['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate', 'personal_status_sex',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker'],
      dtype='object')

Registros de Validación: 
Index(['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate', 'personal_status_sex',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker'],
      dtype='object')


In [138]:
# El dataseto original cuenta con 1,000 registros; después de hacer la partición de los datos, el dataset de entrenamiento cuenta con 850 registros y el de validación con 150 registros.
# Para comprobarlo, utilizamos el método describe en ambos conjuntos (X_train y X_val).
X_train.describe()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,present_residence,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker
count,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00,850.00
mean,2.60,20.46,2.53,2.85,3216.10,2.12,3.38,2.98,2.69,1.16,2.85,2.34,35.60,2.66,1.92,1.40,2.90,1.85,1.40,1.96
std,1.26,11.73,1.08,2.76,2748.34,1.59,1.23,1.11,0.71,0.49,1.10,1.05,11.46,0.72,0.52,0.57,0.65,0.36,0.49,0.19
min,1.00,4.00,0.00,0.00,250.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,19.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
25%,1.00,12.00,2.00,1.00,1374.00,1.00,3.00,2.00,2.00,1.00,2.00,1.00,27.00,3.00,2.00,1.00,3.00,2.00,1.00,2.00
50%,2.00,18.00,2.00,2.00,2301.50,1.00,3.00,3.00,3.00,1.00,3.00,2.00,33.00,3.00,2.00,1.00,3.00,2.00,1.00,2.00
75%,4.00,24.00,4.00,3.00,3959.00,3.00,5.00,4.00,3.00,1.00,4.00,3.00,42.00,3.00,2.00,2.00,3.00,2.00,2.00,2.00
max,4.00,72.00,4.00,10.00,15945.00,5.00,5.00,4.00,4.00,3.00,4.00,4.00,75.00,3.00,3.00,4.00,4.00,2.00,2.00,2.00


In [139]:
X_val.describe()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,present_residence,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker
count,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00,150.00
mean,2.47,23.40,2.65,2.69,3583.73,2.01,3.38,2.93,2.63,1.09,2.80,2.46,35.19,2.74,1.95,1.43,2.93,1.83,1.41,1.96
std,1.24,13.57,1.11,2.65,3204.38,1.52,1.09,1.15,0.67,0.38,1.11,1.05,10.73,0.64,0.57,0.62,0.67,0.38,0.49,0.20
min,1.00,4.00,0.00,0.00,433.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,19.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
25%,1.00,12.00,2.00,1.00,1339.50,1.00,3.00,2.00,2.00,1.00,2.00,2.00,27.00,3.00,2.00,1.00,3.00,2.00,1.00,2.00
50%,2.00,24.00,2.00,2.00,2545.00,1.00,3.00,3.00,3.00,1.00,3.00,3.00,33.00,3.00,2.00,1.00,3.00,2.00,1.00,2.00
75%,4.00,30.00,4.00,3.00,4591.50,3.00,4.00,4.00,3.00,1.00,4.00,3.00,42.00,3.00,2.00,2.00,3.00,2.00,2.00,2.00
max,4.00,60.00,4.00,10.00,18424.00,5.00,5.00,4.00,4.00,3.00,4.00,4.00,74.00,3.00,3.00,4.00,4.00,2.00,2.00,2.00


# **Ejercicio-3.**

Como una primera aproximación (baseline) realizarás las siguientes transformaciones mínimas para generar los primeros modelos. En la misma página de la UCI se indica el tipo de variable de cada uno de los factores. Definen cuatro tipo de variables: categórica (categorical), ordinal (discretized quantitative), numérica (quantitative) y binaria (binary). Con base a dicha información realiza un Pipeline que incluya al menos las siguientes transformaciones:
1. Imputación a todas las variables de entrada, diferenciando entre el tipo de cada variable (decide y justifica que tipo de imputación realizas en cada caso).
2. Realiza un análisis de las variables numéricas (quantitative) de entrada y aplica una transformación que escale a todas ellas en un rango equiparable.
3. Aplica la transformación One-Hot encoding a las variables de entrada de tipo categórico y binaria. En particular, justifica por qué una variable binaria requeriría que se le aplique la transformación one-hot encoding. Por el momento dejar las variables ordinales sin transformar.

In [140]:
# Primero, dividimos el dataframe original en cuatro dataframes, uno por cada tipo de varaible, según la descripción de UCI.

# Variables categóricas (categorical)
df_categorical = ['status','credit_history','purpose','savings','personal_status_sex','other_debtors','other_installment_plans','housing']

# Variables numéricas ordinales/discretas (ordinal; discretized quantitative)
df_discretized_quantitative = ['employment_duration','installment_rate','present_residence','property','number_credits','job', 'people_liable']

# Variables numéricas continuas (quantitative)
df_continuous_quantitative = ['duration','amount','age']

# Variables binarias (binary)
# Estoy dejando fuera 'credit_risk' porque es la variable objetivo.
df_binary = ['telephone','foreign_worker']

In [141]:
# Para realizar imputación de datos faltantes, primero debemos revisar si tenemos datos faltantes en nuestro dataset.
# Para ello, utilizamos el método isnull() y sumamos los valores de cada columna.
df.isnull().sum()

status                     0
duration                   0
credit_history             0
purpose                    0
amount                     0
savings                    0
employment_duration        0
installment_rate           0
personal_status_sex        0
other_debtors              0
present_residence          0
property                   0
age                        0
other_installment_plans    0
housing                    0
number_credits             0
job                        0
people_liable              0
telephone                  0
foreign_worker             0
credit_risk                0
dtype: int64

 Según vemos en el resultado, no tenemos datos faltantes en nuestro dataset.

In [142]:
# El siguiente paso del ejercicio es escalar los datos numéricos, para que todos los valores estén en el mismo rango.
# Para ello, utilizamos el método StandardScaler() de la librería sklearn.preprocessing.

# Primero, debemos agrupar las variables numéricas en un solo dataframe, pero, como solo aplicaremos las transformaciones al conjunto de entrenamiento, entonces solo dividiremos las variables numéricas del conjunto de entrenamiento.
X_train_quantitative = X_train[df_discretized_quantitative + df_continuous_quantitative]

# Ahora, aplicamos la transformación StandardScaler() a las variables numéricas del conjunto de entrenamiento.
X_train_quantitative_scaled = StandardScaler().fit_transform(X_train_quantitative)
X_train_quantitative.head()

,employment_duration,installment_rate,present_residence,property,number_credits,job,people_liable,duration,amount,age
548,3,4,1,1,1,3,2,12,1007,22
335,2,4,1,2,1,2,2,24,1747,24
793,2,4,4,2,1,3,2,12,795,53
848,3,4,4,4,1,3,2,36,2671,50
295,3,1,2,2,1,3,2,36,8133,30


In [143]:
# Después, para aplicar la transformación One-Hot Encoder a las variables categóricas del conjunto de entrenamiento, primero debemos agrupar las variables categóricas en un solo dataframe.
X_train_catbin = X_train[df_categorical + df_binary]
X_train_catbin.head()

,status,credit_history,purpose,savings,personal_status_sex,other_debtors,other_installment_plans,housing,telephone,foreign_worker
548,2,2,0,4,4,1,3,2,1,2
335,1,2,2,1,3,2,3,2,1,1
793,1,2,6,1,2,1,3,2,1,2
848,2,2,3,2,2,2,3,3,1,2
295,4,2,1,1,2,1,1,2,1,2


In [144]:
# Ahora, podemos aplicar la transformación One-Hot Encoder al dataframe X_train_catbin.
X_train_catbin_encoded = pd.get_dummies(X_train_catbin)
X_train_catbin_encoded.head()

X_train_catbin_pipeline = pipeline.Pipeline(steps = [('imp_mostfrequent', SimpleImputer(strategy='most_frequent')),('OneHotE', OneHotEncoder(handle_unknown="ignore"))])

X_train_catbin_pipeline.fit(X_train_catbin)
X_train_catbin_encoded = X_train_catbin_pipeline.transform(X_train_catbin)
pd.DataFrame(X_train_catbin_encoded.toarray(), columns = X_train_catbin_pipeline['OneHotE'].get_feature_names()).head(10)

c:\Users\ferna\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,x0_1,x0_2,x0_3,x0_4,x1_0,x1_1,x1_2,x1_3,x1_4,x2_0,...,x6_1,x6_2,x6_3,x7_1,x7_2,x7_3,x8_1,x8_2,x9_1,x9_2
0,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,...,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
1,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00
2,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
3,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00
4,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,...,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
5,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,...,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
6,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,...,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
7,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00
8,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
9,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,...,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00


# **Ejercicio-4.**

Llevarás un entrenamiento usando validación cruzada entre los siguientes tres modelos de aprendizaje automático: Regresión Logística, Árbol de Decisión y Bosque Aleatorio. Deberás 
llevar a cabo el entrenamiento de los tres de manera conjunta usando un ciclo FOR (*for loop*). Recuerda aplicar las transformaciones que definiste en tu Pipeline. El entrenamiento debe ser con las siguientes características:
1. Usa los parámetros predeterminados de cada modelo.
2. En cada iteración deben calcularse todas las siguientes métricas: accuracy, precision, recall, f1-score y Gmean. Todas estas métricas deben ser funciones que tú mismo debes definir (Es decir, no usar las funciones de dichas métricas que te proporciona scikit-learn. Sin embargo, sí puedes usar la información regresada por el método confusion_matrix() de scikit-learn para definir las métricas).
3. Usar validación cruzada estratificada con 5 particiones y con 3 repeticiones.
4. Imprimir el valor de todas estas métricas, tanto para los datos de entrenamiento, como para los de validación. Así como los diagramas de caja y bigotes de los tres modelos con la métrica “recall”. ¿Alguno de los modelos está subentrenado o sobreentrenado? Justifica tu respuesta.
5. En particular obtengamos algunas de las llamadas curvas de aprendizaje para algunos de estos casos. En dada gráfico debes incluir tus comentarios sobre el modelo generado:
   1. Obtener las curvas de aprendizaje (*learning_curve*) en la cual se va incrementando el tamaño de la muestra para el modelo de regresión Logística con su hiperparámetros predeterminados Utilizar al menos 20 puntos en la partición de los conjuntos de entrenamiento y la métrica “*f1-score*”, como evaluación del desempeño de dicha función “*learning_curve()*”.
   2. Obtener las curvas de validación (validation_curve) en la cual se va incrementando la complejidad del hiperparámetro “max_depth” para el modelo de árbol de decisión con sus hiperparámetros predeterminados. Utilizar valores de máxima profundidad desde 1 hasta 20 y con la métrica “f1-score” para la evaluación del desempeño del modelo.
   3. Obtener las curvas de aprendizaje (learning_curve) en la cual se va incrementando el tamaño de la muestra para el modelo de regresión bosque aleatorio (random forest) con su hiperparámetros predeterminados. Utilizar al menos 20 puntos en la partición de los conjuntos de entrenamiento y la métrica “recall”, como evaluación del desempeño del modelo.

In [145]:
# Similar a como lo vimos en actividades anteriores, iniciamos definiendo una función llamada get_models, en la cual creamos dos listas para ir agregando los modelos y sus nombres.
def get_models():
  modelos = list()
  nombres = list()

# Luego creamos cada uno de los modelos solicitados:

  # LinReg - Regresión Logística:
  modelos.append(LogisticRegression(solver='liblinear'))
  nombres.append('LinReg')

  # DecTree - Árbol de Decisión:
  modelos.append(DecisionTreeClassifier())
  nombres.append('DecTree')
  
  #RandForest - Ranfom Forest
  modelos.append(RandomForestClassifier())
  nombres.append('RandForest')
  
  return modelos, nombres

In [146]:
def mi_accuracy(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel()
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  return accuracy

def mi_precision(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel()
  precision = tp/(tp+fp)
  return precision

def mi_recall(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel()
  recall = tp/(tp+fn)
  return recall

def mi_fscore(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel()
  fscore = (2*tp)/((2*tp)+fp+fn)
  return fscore

def mi_gmean(yreal, ypred):
  tn,fp,fn,tp= confusion_matrix(yreal,ypred).ravel()
  recall = mi_recall(yreal, ypred)
  especificidad= tn / (tn+fp)
  gmean = np.sqrt(recall*especificidad)
  return gmean

# **Ejercicio-5.**

Finalmente veamos la manera de mejorar los valores de los hiperparámetros de cada modelo, así como el problema del sobreentrenamiento de algunos de ellos. Para ello deberás usar el método GridSearchCV() de scikit-learn. Recuerda que este método hace una búsqueda de los mejores hiperparámetros de un modelo mediante el llamada formato de malla y aplicando validación cruzada. En cada caso puedes incrementar el máximo de iteraciones,”max_iter” para que tengas la convergencia adecuada para todas las combinaciones en cada modelo. Recuerda también aplicar las transformaciones que definiste en tu Pipeline. Para fines de este ejercicio se ha seleccionado para cada modelo una métrica diferente, que permita irte familiarizando con ellas. Puedes consultar su documentación de GridSearchCV en la siguiente liga:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html 

1. Para el modelo de regresión logística realizar el entrenamiento buscando sus mejores hiperparámetros con GridSearchCV(). Los hiperparámetros que debes incluir en su búsqueda deben ser al menos los siguientes: C, solver, class_weight y penalty. En este caso deberás usar la métrica (scoring) “f1-score”. Imprime la mejor combinación de parámetros obtenidos, así como el valor del mejor desempeño (score) obtenido con la métrica f1. ¿Cuál es la utilidad de la métrica “f1-score”? Incluye tus conclusiones. NOTA: Toma en cuenta que no todas las combinaciones de “solver” y “penalty” son posibles, para que lo tomes en cuenta al momento de realizar la búsqueda. Revisa la documentación.
2. Con los mejores valores de los hiperparámetros encontrados con la métrica “f1-score” para el modelo de regresión logística, obtener las curvas de aprendizaje (learning curve), incrementando el tamaño del conjunto de entrenamiento al menos 20 veces. Si lo crees adecuado, puedes hacer los ajustes que consideres adecuados para mejorar el resultado y evitar el sobreentrenamiento o el subentrenamiento.
3. Para el modelo de árbol de decisión (decision tree) realizar el entrenamiento buscando sus mejores hiperparámetros con GridSearchCV(). Los hiperparámetros que debes incluir en su búsqueda deben ser al menos los siguientes: ccp_alpha, criterion, max_depth, min_samples_split y class_weight. En este caso deberás usar la métrica (scoring) “precision”. Imprime la mejor combinación de parámetros obtenidos, así como el valor del mejor desempeño (score) obtenido con la métrica “precision”. ¿Cuál es la utilidad de la métrica “precision”? Incluye tus conclusiones.
4. Con los mejores valores de los hiperparámetros encontrados con la métrica “precision” para el modelo de árbol de decisión, obtener las curvas de aprendizaje (learning curve), incrementando el tamaño del conjunto de entrenamiento al menos 20 veces. Si lo crees adecuado, puedes hacer los ajustes que consideres adecuados para mejorar el resultado y evitar el sobreentrenamiento o el subentrenamiento.
5. Para el modelo de bosque aleatorio (random forest) realizar el entrenamiento buscando sus mejores hiperparámetros con GridSearchCV(). Los hiperparámetros que debes incluir en su búsqueda deben ser al menos los siguientes: ccp_alpha, criterion, max_depth, min_samples_split y class_weight. En este caso deberás usar la métrica (scoring) “recall”. Imprime la mejor combinación de parámetros obtenidos, así como el valor del mejor desempeño (score) obtenido con la métrica “recall”. ¿Cuál es la utilidad de la métrica “recall”? Incluye tus conclusiones. NOTA: Toma en cuenta que el método de random forest pude tardar varios minutos en llevar a cabo.
6. Con los mejores valores de los hiperparámetros encontrados con la métrica “recall” para el modelo de bosque aleatorio, obtener las curvas de validación (validation curve), incrementando la complejidad del modelo a través del hiperparámetro “max_depth” con al menos 10 valores. Si lo crees adecuado, puedes hacer los ajustes que consideres adecuados para mejorar el resultado y evitar el sobreentrenamiento o el subentrenamiento. 

In [147]:
None

# **Ejercicio-6.**

Para cada uno de estos tres modelos, con las métricas que se consideraron en cada caso y usando el conjunto de Prueba que no has utilizado hasta ahora, obtener los modelos finales como se te 
indica a continuación. Deberás usar además como conjunto de entrenamiento el llamado modelo de entrenamiento “aumentado” que consiste en las datos que estuviste utilizando para entrenamiento y validación:
1. Obtener el  modelo de regresión logística con los mejores parámetros que hayas encontrado con la métrica f1-score utilizada. Imprimir el valor de dicha métrica e incluye tus conclusiones finales para este caso. Incluir un gráfico del árbol de decisión final obtenido.
2. Obtener el  modelo de árbol de decisiones con los mejores parámetros que hayas encontrado con la métrica “precision” utilizada. Imprimir el valor de dicha métrica e incluye tus conclusiones finales para este caso.
3. Obtener el  modelo de bosque aleatorio con los mejores parámetros que hayas encontrado con la métrica “recall” utilizada. Imprimir el valor de dicha métrica e incluye tus conclusiones finales para este caso. 

In [148]:
None

### **Fin de la Actividad de la semana 6.**

---